In [ ]:
%pylab inline

In [ ]:
from IPython.display import HTML

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
import katdal
import pickle
import scikits.fitting as fit
import katpoint
from katsdpcal import calprocs
import logging
from katsdpscripts import git_info
import scape

In [ ]:
pickle_file = open('/var/kat/katsdpscripts/RTS/rfi_mask.pickle')
rfi_static_flags = pickle.load(pickle_file)
pickle_file.close()
edge = np.tile(True,4096)
edge[slice(211,3896)] = False
static_flags = np.logical_or(edge,rfi_static_flags)

In [ ]:
filename = 

In [ ]:
data = katdal.open(filename)

In [ ]:
file_base = filename.split('/')[-1].split('.')[0]
nice_filename =  file_base + '_T_sys_T_nd'
logger = logging.root
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(nice_filename + '.log', 'w')
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter('%(levelname)s: %(message)s'))
logger.addHandler(fh)
logger.info('Beginning data processing with:\n%s'%git_info('standard'))


# Data details

In [ ]:
print data

In [ ]:
def activity(h5,state = 'track'):
    """Activity Sensor because some of antennas have a mind of their own, 
    others appear to have lost theirs entirely """
    antlist = [a.name for a in h5.ants]
    activityV = np.zeros((len(antlist),h5.shape[0]) ,dtype=np.bool)
    for i,ant in enumerate(antlist) :
        sensor = h5.sensor['Antennas/%s/activity'%(ant)]
        activityV[i,:] +=   (sensor==state)
    return np.all(activityV,axis=0)

def w_average(arr,axis=None, weights=None):
    return np.nansum(arr*weights,axis=axis)/np.nansum(weights,axis=axis)

def reduce_compscan_inf(h5 ,channel_mask = None,chunks=16,return_raw=False):
    """Break the band up into chunks"""
    chunk_size = chunks
    rfi_static_flags = np.tile(False, h5.shape[0])
    if len(channel_mask)>0:
        pickle_file = open(channel_mask)
        rfi_static_flags = pickle.load(pickle_file)
        pickle_file.close()
    gains_p = {}
    stdv = {}
    calibrated = False # placeholder for calibration
    #if not return_raw:     # Calculate average target flux over entire band
    target = h5.catalogue.targets[h5.target_indices[0]]
    flux_spectrum = h5.catalogue.targets[h5.target_indices[0]].flux_density(h5.freqs) # include flags
    average_flux = np.mean([flux for flux in flux_spectrum if not np.isnan(flux)])
    temperature = np.mean(h5.temperature)
    pressure = np.mean(h5.pressure)
    humidity = np.mean(h5.humidity)
    wind_speed = np.mean(h5.wind_speed)
    wind_direction  = np.degrees(np.angle(np.mean(np.exp(1j*np.radians(h5.wind_direction)))) )# Vector Mean
    sun = katpoint.Target('Sun, special')
    # Calculate pointing offset
    # Obtain middle timestamp of compound scan, where all pointing calculations are done
    middle_time = np.median(h5.timestamps[:], axis=None)
    # Start with requested (az, el) coordinates, as they apply at the middle time for a moving target
    requested_azel = target.azel(middle_time)
    # Correct for refraction, which becomes the requested value at input of pointing model
    rc = katpoint.RefractionCorrection()
    requested_azel = [requested_azel[0], rc.apply(requested_azel[1], temperature, pressure, humidity)]
    requested_azel = katpoint.rad2deg(np.array(requested_azel))

   
    gaussian_centre     = np.zeros((chunk_size* 2,2,len(h5.ants)) )
    gaussian_centre_std = np.zeros((chunk_size* 2,2,len(h5.ants)) )
    gaussian_width      = np.zeros((chunk_size* 2,2,len(h5.ants)) )
    gaussian_width_std  = np.zeros((chunk_size* 2,2,len(h5.ants)) )
    gaussian_height     = np.zeros((chunk_size* 2,len(h5.ants)) )
    gaussian_height_std = np.zeros((chunk_size* 2,len(h5.ants)) )
    h5.antlist = [a.name for a in h5.ants]
    h5.bls_lookup = calprocs.get_bls_lookup(h5.antlist,h5.corr_products)
    pols = ["H","V"] # Put in logic for Intensity
    for i,pol in enumerate(pols) :
        gains_p[pol] = []
        pos = []
        stdv[pol] = []
        h5.select(pol=pol)
        h5.bls_lookup = calprocs.get_bls_lookup(h5.antlist,h5.corr_products)
        for scan in h5.scans() : 
            valid_index = activity(h5,state = 'track')
            data = h5.vis[valid_index]
            if data.shape[0] > 0 : # need at least one data point
                gains_p[pol].append(calprocs.g_fit(data[:,:,:].mean(axis=0),h5.bls_lookup,refant=0) )
                stdv[pol].append(np.ones((data.shape[0],data.shape[1],len(h5.ants))).sum(axis=0))#number of data points
                # Get coords in (x(time,ants),y(time,ants) coords) 
                pos.append( [h5.target_x[valid_index,:].mean(axis=0), h5.target_y[valid_index,:].mean(axis=0)] ) 
        for ant in range(len(h5.ants)):
            for chunk in range(chunks):
                if np.array(pos).shape[0] > 4 : # Make sure there is enough data for a fit
                    freq = slice(chunk*(h5.shape[1]//chunks),(chunk+1)*(h5.shape[1]//chunks))
                    rfi = ~rfi_static_flags[freq]   
                    fitobj  = fit.GaussianFit(np.array(pos)[:,:,ant].mean(axis=0),[1.,1.],1)
                    x = np.column_stack((np.array(pos)[:,0,ant],np.array(pos)[:,1,ant]))
                    y = np.abs(np.array(gains_p[pol])[:,freq,:][:,rfi,ant]).mean(axis=1)
                    y_err = 1./np.sqrt(np.array(stdv[pol])[:,freq,:][:,rfi,ant].sum(axis=1))
                    gaussian = fitobj.fit(x.T,y,y_err ) 
                    #Fitted beam center is in (x, y) coordinates, in projection centred on target
                    snr = np.abs(np.r_[gaussian.std/gaussian.std_std])
                    valid_fit = np.all(np.isfinite(np.r_[gaussian.mean,gaussian.std_mean,gaussian.std,gaussian.std_std,gaussian.height,gaussian.std_height,snr]))
                    theta =  np.sqrt((gaussian.mean**2).sum())  # this is to see if the co-ord is out of range
                    #The valid fit is needed because I have no way of working out if the gain solution was ok.
                    if  not valid_fit or np.any(theta > np.pi) : # the checks to see if the fit is ok
                        gaussian_centre[chunk+i*chunk_size,:,ant]     =  np.nan
                        gaussian_centre_std[chunk+i*chunk_size,:,ant] =  np.nan
                        gaussian_width[chunk+i*chunk_size,:,ant]      =  np.nan
                        gaussian_width_std[chunk+i*chunk_size,:,ant]  =  np.nan
                        gaussian_height[chunk+i*chunk_size,ant]       =  np.nan
                        gaussian_height_std[chunk+i*chunk_size,ant]   =  np.nan
                    else:
                        # Convert this offset back to spherical (az, el) coordinates
                        beam_center_azel = target.plane_to_sphere(np.radians(gaussian.mean[0]), np.radians(gaussian.mean[1]), middle_time)
                        # Now correct the measured (az, el) for refraction and then apply the old pointing model
                        # to get a "raw" measured (az, el) at the output of the pointing model
                        beam_center_azel = [beam_center_azel[0], rc.apply(beam_center_azel[1], temperature, pressure, humidity)]
                        beam_center_azel = h5.ants[ant].pointing_model.apply(*beam_center_azel)
                        beam_center_azel = np.degrees(np.array(beam_center_azel))
                        gaussian_centre[chunk+i*chunk_size,:,ant]     = beam_center_azel
                        gaussian_centre_std[chunk+i*chunk_size,:,ant] = gaussian.std_mean
                        gaussian_width[chunk+i*chunk_size,:,ant]      = gaussian.std
                        gaussian_width_std[chunk+i*chunk_size,:,ant]  = gaussian.std_std
                        gaussian_height[chunk+i*chunk_size,ant]       = gaussian.height
                        gaussian_height_std[chunk+i*chunk_size,ant]   = gaussian.std_height

    if return_raw :
        return np.r_[gaussian_centre , gaussian_centre_std , gaussian_width , gaussian_width_std , gaussian_height , gaussian_height_std]
    else:
        ant_pointing = {}
        pols = ["HH","VV",'I']
        pol_ind = {}
        pol_ind['HH'] = np.arange(0.0*chunk_size,1.0*chunk_size,dtype=int)
        pol_ind['VV'] = np.arange(1.0*chunk_size,2.0*chunk_size,dtype=int) 
        pol_ind['I']  = np.arange(0.0*chunk_size,2.0*chunk_size,dtype=int) 
        for ant in range(len(h5.ants)):
            if np.any(np.isfinite(w_average(gaussian_centre[:,:,ant],axis=0,weights=1./gaussian_centre_std[:,:,ant]**2)) ) : # a bit overboard
                name = h5.ants[ant].name
                ant_pointing[name] = {}
                ant_pointing[name]["antenna"] = h5.ants[ant].name
                ant_pointing[name]["dataset"] = h5.name.split('/')[-1].split('.')[0]
                ant_pointing[name]["target"] = target.name
                ant_pointing[name]["timestamp_ut"] =str(katpoint.Timestamp(middle_time))
                ant_pointing[name]["data_unit"] = 'Jy' if calibrated else 'counts'
                ant_pointing[name]["frequency"] = h5.freqs.mean()
                ant_pointing[name]["flux"] = average_flux
                ant_pointing[name]["temperature"] =temperature
                ant_pointing[name]["pressure"] =pressure
                ant_pointing[name]["humidity"] =humidity
                ant_pointing[name]["wind_speed"] =wind_speed
                ant_pointing[name]["wind_direction"] =wind_direction
                # work out the sun's angle
                sun_azel = katpoint.rad2deg(np.array(sun.azel(middle_time,antenna=h5.ants[ant])))  
                ant_pointing[name]["sun_az"] = sun_azel.tolist()[0]
                ant_pointing[name]["sun_el"] = sun_azel.tolist()[1]
                ant_pointing[name]["timestamp"] =middle_time.astype(int)
                #Work out the Target position and the requested position
                # Start with requested (az, el) coordinates, as they apply at the middle time for a moving target
                requested_azel = target.azel(middle_time,antenna=h5.ants[ant])
                # Correct for refraction, which becomes the requested value at input of pointing model
                rc = katpoint.RefractionCorrection()
                requested_azel = [requested_azel[0], rc.apply(requested_azel[1], temperature, pressure, humidity)]
                requested_azel = katpoint.rad2deg(np.array(requested_azel))
                target_azel = katpoint.rad2deg(np.array(target.azel(middle_time,antenna=h5.ants[ant])))  
                ant_pointing[name]["azimuth"] =target_azel.tolist()[0]
                ant_pointing[name]["elevation"] =target_azel.tolist()[1]
                azel_beam = w_average(gaussian_centre[pol_ind["I"],:,ant],axis=0,weights=1./gaussian_centre_std[pol_ind["I"],:,ant]**2)
                # Make sure the offset is a small angle around 0 degrees
                offset_azel = katpoint.wrap_angle(azel_beam - requested_azel, 360.)
                ant_pointing[name]["delta_azimuth"] =offset_azel.tolist()[0]
                ant_pointing[name]["delta_elevation"] =offset_azel.tolist()[1]
                ant_pointing[name]["delta_elevation_std"] =0.0#calc
                ant_pointing[name]["delta_azimuth_std"] =0.0#calc
                for pol in pol_ind:
                    ant_pointing[name]["beam_height_%s"%(pol)]     = w_average(gaussian_height[pol_ind[pol],ant],axis=0,weights=1./gaussian_height_std[pol_ind[pol],ant]**2)
                    ant_pointing[name]["beam_height_%s_std"%(pol)] = np.sqrt(np.nansum(1./gaussian_height_std[pol_ind[pol],ant]**2) )
                    ant_pointing[name]["beam_width_%s"%(pol)]      = w_average(gaussian_width[pol_ind[pol],:,ant],axis=0,weights=1./gaussian_width_std[pol_ind[pol],:,ant]**2).mean() 
                    ant_pointing[name]["beam_width_%s_std"%(pol)]  = np.sqrt(np.nansum(1./gaussian_width_std[pol_ind[pol],:,ant]**2) )
                    ant_pointing[name]["baseline_height_%s"%(pol)] = 0.0
                    ant_pointing[name]["baseline_height_%s_std"%(pol)] = 0.0
                    ant_pointing[name]["refined_%s"%(pol)] =  5.0  # I don't know what this means 
                    ant_pointing[name]["azimuth_%s"%(pol)]       =w_average(gaussian_centre[pol_ind[pol],0,ant],axis=0,weights=1./gaussian_centre_std[pol_ind[pol],0,ant]**2)
                    ant_pointing[name]["elevation_%s"%(pol)]     =w_average(gaussian_centre[pol_ind[pol],1,ant],axis=0,weights=1./gaussian_centre_std[pol_ind[pol],1,ant]**2)
                    ant_pointing[name]["azimuth_%s_std"%(pol)]   =np.sqrt(np.nansum(1./gaussian_centre_std[pol_ind[pol],0,ant]**2) )
                    ant_pointing[name]["elevation_%s_std"%(pol)] =np.sqrt(np.nansum(1./gaussian_centre_std[pol_ind[pol],1,ant]**2) )

        return ant_pointing


In [ ]:
def plot_bpcal_selection(f,corrprods='cross',compscans=[0], scans='track',channels=~static_flags):
    fig = plt.figure(figsize=(15,15))
    plt.suptitle("Phaseup for %s"%data.start_time.local(),fontsize=16, fontweight="bold")    
    try:
        for pol in 'hv':
            f.select(corrprods='cross',compscans=compscans, scans=scans,pol=pol,channels=channels)
            ts = data.timestamps - data.timestamps[0]
            if f.shape[0] == 0:
                raise ObsReporterError('The selection criteria resulted in an empty data set.')
            crosscorr = [(f.inputs.index(inpA), f.inputs.index(inpB)) for inpA, inpB in f.corr_products]
            #extract the fringes
            vis = f.vis[:,:,:]
            #For plotting the fringes
            fig.subplots_adjust(wspace=0., hspace=0.)
            #debug_here()
            for n, (indexA, indexB) in enumerate(crosscorr):
                subplot_index = (len(f.ants) * indexA + indexB + 1) if pol == 'h' else (indexA + len(f.ants) * indexB + 1)
                ax = fig.add_subplot(len(f.ants), len(f.ants), subplot_index)
                ax.imshow(angle(vis[:,:,n]),aspect=float(vis.shape[1])/vis.shape[0])
                ax.set_xticks([])
                ax.set_yticks([])
                if pol == 'h':
                    if indexA == 0:
                        ax.xaxis.set_label_position('top')
                        ax.set_xlabel(f.inputs[indexB][:],size='xx-large')
                    if indexB == len(f.ants) - 1:
                       ax.yaxis.set_label_position('right')
                       ax.set_ylabel(f.inputs[indexA][:], rotation='horizontal',size = 'xx-large')
                else:
                    if indexA == 0:
                        ax.set_ylabel(f.inputs[indexB][:], rotation='horizontal',size='xx-large')
                    if indexB == len(f.ants) - 1:
                        ax.set_xlabel(f.inputs[indexA][:],size='xx-large')
    except ObsReporterError, error:
           print 'Failed with selection: f.shape=%s. Error: %s' % (str(f.shape), error)
    except KeyError, error:
            print 'Failed to read scans from File: %s with Key Error: %s' % (f, error)
    except ValueError, error:
            print 'Failed to read scans from File: %s with Value Error: %s' % (f, error)


# Phase up results

In [ ]:
data.select()
plot_bpcal_selection(data,corrprods='cross',compscans=['calibration'], scans='track',channels=~static_flags)

# Tsys results

In [ ]:
figure(1,figsize=(15,5))
data.select()
ants = data.ants
for pol in 'hv':
    for ant in ants:
        try:
            rx_sn = data.receivers[ant]
        except KeyError:
            logger.error('Receiver serial number for antennna %s not found in the H5 file'%ant.name)
            rx_sn = 'SN_NOT_FOUND'
        diode_filename = '/var/kat/katconfig/user/noise-diode-models/mkat/rx.'+rx_sn+'.'+pol+'.csv'
        logger.info('Loading noise diode file %s from config'%diode_filename)
        try:
            nd = scape.gaincal.NoiseDiodeModel(diode_filename)
        except:
            logger.error("Error reading the noise diode file ... using a constant value of 20k")
            logger.error("Be sure to reprocess the data once the file is in the config")
            nd = scape.gaincal.NoiseDiodeModel(freq=[856,1712],temp=[20,20])
        data.select(ants=ant.name,corrprods='auto',pol=pol,channels=~static_flags,compscans=['coupler'])
        figure(2,figsize=(15,5))
        d = scape.DataSet(data)
        scape.plot_xyz(d,'time','amp',label='Average of the data')
        on = data.sensor['Antennas/'+ant.name+'/nd_coupler']
        ts = data.timestamps - data.timestamps[0]
        plt.plot(ts,np.array(on).astype(float)*4000,'g',label='katdal ND sensor')
        plt.title("Timeseries for all antennas - %s"%(git_info()))
        figure(1)
        freq = data.channel_freqs
        nd_temp = nd.temperature(freq / 1e6)
        vis = data.vis[:]
        on = data.sensor['Antennas/'+ant.name+'/nd_coupler']
        on[0] = False
        n_on = np.tile(False,on.shape[0])
        n_off = np.tile(False,on.shape[0])
        buff = 1
        if not any(on):
            logger.critical('No noise diode fired during track of')
        else:
            jumps = (np.diff(on).nonzero()[0] + 1).tolist()
            n_on[slice(jumps[0]+buff,jumps[1]-buff)] = True
            n_off[slice(jumps[1]+buff,-buff)] = True
        spec = np.mean(vis[n_off,:,0],0)
        nd_spec = np.mean(vis[n_on,:,0],0)
        jump = nd_spec - spec
        Tsys = spec * nd_temp/(jump)
        p = 1 if pol == 'h' else 2
        p_title = 'H Pol' if pol == 'h' else 'V Pol'
        subplot(1,2,p)
        norm = Normalize(0,64)
        color = cm.brg(norm(int(ant.name[1:])))
        ax = gca()
        plot(freq/1e6,Tsys,'.',color=color,label=ant.name)
        ylim(10,30)
        grid()
        title(p_title)
        ylabel('Tsys/[K]')
        xlabel('freq/[MHz]')
        
fig = pyplot.figure(0,figsize=(15, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cmap = mpl.cm.brg
norm = mpl.colors.Normalize(vmin=0, vmax=63)
cb1 = mpl.colorbar.ColorbarBase(ax1,cmap=cmap,norm=norm,orientation='horizontal')
cb1.set_label('Antenna colors')

# Pointing results

In [ ]:
data.select()
data.select(compscans=['interferometric_pointing'],channels=~static_flags)
res = reduce_compscan_inf(data,"/var/kat/katsdpscripts/RTS/rfi_mask.pickle")

In [ ]:
figure(0,figsize=(15,5))
data.select()
ants = data.ants
for ant in ants:
    data.select(ants=ant.name,compscans=['raster'], scans='scan',channels=~static_flags)
    for c in data.compscans():
        norm = Normalize(0,64)
        color = cm.brg(norm(int(ant.name[1:])))
        d = scape.DataSet(data)
        for i in range(len(d.scans)):
            d.scans[i].data = scape.stats.remove_spikes(d.scans[i].data,axis=1,spike_width=3,outlier_sigma=5.)    
        d.average()
        d.fit_beams_and_baselines()
        compscan = d.compscans[0]
        if compscan.beam == None:
            continue
        figure(figsize=(15,5))
        #scape.plot_compound_scan_on_target(compscan)
        scape.plot_xyz(compscan,'target_x','target_y','amp',color=color,labels='')
        ax = gca()
        el = matplotlib.patches.Ellipse((0,0), 5./60, 5./60, facecolor='y', alpha=0.9)
        ax.add_artist(el)
        plot([rad2deg(compscan.beam.center[0])], [rad2deg(compscan.beam.center[1])],
                'ko',label='Single Dish')
        title('Antenna: %s Beamfit valid: %s'%(ant.name,str(compscan.beam.is_valid)))
        
        target = data.catalogue.targets[data.target_indices[0]]
        temperature = np.mean(data.temperature)
        pressure = np.mean(data.pressure)
        humidity = np.mean(data.humidity)
        middle_time = np.median(data.timestamps[:], axis=None)
        # Start with requested (az, el) coordinates, as they apply at the middle time for a moving target
        requested_azel = katpoint.rad2deg(array(target.azel(middle_time)))
        # Correct for refraction, which becomes the requested value at input of pointing model
        rc = katpoint.RefractionCorrection()
    
        beam_center_xy = compscan.beam.center
        # Convert this offset back to spherical (az, el) coordinates
        beam_center_azel = compscan.target.plane_to_sphere(beam_center_xy[0], beam_center_xy[1], middle_time)
        # Now correct the measured (az, el) for refraction and then apply the old pointing model
        # to get a "raw" measured (az, el) at the output of the pointing model
        beam_center_azel = [beam_center_azel[0], rc.apply(beam_center_azel[1], temperature, pressure, humidity)]
        beam_center_azel = compscan.dataset.antenna.pointing_model.apply(*beam_center_azel)
        beam_center_azel = katpoint.rad2deg(np.array(beam_center_azel))
        # Make sure the offset is a small angle around 0 degrees
        offset_azel = scape.stats.angle_wrap(beam_center_azel - requested_azel, 360.)
        
        
        int_offset_azel = [res[ant.name]['delta_azimuth'],res[ant.name]['delta_elevation']]
        int_beam_center_azel = scape.stats.angle_wrap(int_offset_azel + requested_azel, 360.)
        int_beam_center_azel = katpoint.deg2rad(np.array(int_beam_center_azel))
        int_beam_center_azel = compscan.dataset.antenna.pointing_model.reverse(*int_beam_center_azel)
        int_beam_center_azel = [int_beam_center_azel[0], rc.reverse(int_beam_center_azel[1], temperature, pressure, humidity)]
        int_beam_center_xy = compscan.target.sphere_to_plane(int_beam_center_azel[0], int_beam_center_azel[1], middle_time)
        plot([rad2deg(int_beam_center_xy[0])], [rad2deg(int_beam_center_xy[1])],
                'k*',label='Interferometric')
        legend()
        grid()
        print 'Azimith: ',ant.name,offset_azel[0],res[ant.name]['delta_azimuth']
        print 'Elevation: ',ant.name,offset_azel[1],res[ant.name]['delta_elevation']
        print "---"

fig = pyplot.figure(0,figsize=(15, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cmap = mpl.cm.brg
norm = mpl.colors.Normalize(vmin=0, vmax=63)
cb1 = mpl.colorbar.ColorbarBase(ax1,cmap=cmap,norm=norm,orientation='horizontal')
cb1.set_label('Antenna colors')

In [ ]:
print 'Report generated:'
!date
print '-----------------'
print git_info(['scape','katdal','katpoint','katsdpscripts'])